#### 0. Import Librerie

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
import plotly.graph_objects as go


#### 1. Preparazione Dati

In [10]:
######################
#   UPLOAD DATI
######################
df15 = pd.read_csv('/Users/federicogamberini/VS Code/when-to-buy-iphone/Dataset/iPhone15.csv')
df16 = pd.read_csv('/Users/federicogamberini/VS Code/when-to-buy-iphone/Dataset/iPhone16.csv')
df15['Data'] = pd.to_datetime(df15['Data'])
df16['Data'] = pd.to_datetime(df16['Data'])
df15.head(), df16.head()

(        Data  Prezzo  Giorni_dal_lancio Trimestre   Modello
 0 2023-09-16   979.0                  0        T1  iPhone15
 1 2023-09-17   979.0                  1        T1  iPhone15
 2 2023-09-18   979.0                  2        T1  iPhone15
 3 2023-09-19   979.0                  3        T1  iPhone15
 4 2023-09-20   979.0                  4        T1  iPhone15,
         Data  Prezzo  Giorni_dal_lancio Trimestre   Modello
 0 2024-09-13   979.0                  0        T1  iPhone16
 1 2024-09-14   979.0                  1        T1  iPhone16
 2 2024-09-15   979.0                  2        T1  iPhone16
 3 2024-09-16   979.0                  3        T1  iPhone16
 4 2024-09-17   979.0                  4        T1  iPhone16)

In [11]:
######################
#   PREPARAZIONE DATI
######################
# Traslare la data di iPhone 15 di 1 anno avanti
df15['Data_shifted'] = df15['Data'] + pd.DateOffset(years=1)

# unione dei i due dataframe sulla data traslata di iphone 15 e sulla data del 16
merged = pd.merge(
    df16, 
    df15[['Data_shifted', 'Prezzo']], 
    left_on='Data', 
    right_on='Data_shifted',
    how='inner',
    suffixes=('_16', '_15')
)

merged = merged.drop(columns=['Data_shifted', 'Modello'])
print(merged.head())

# Correlazione tri i prezzi di iphone 16 e 15 shiftato, per vedere se c'è una relazione
correlation = merged['Prezzo_16'].corr(merged['Prezzo_15'])
print(f'\nCorrelazione iPhone 16 vs iPhone 15 shiftato: {correlation:.3f}.')
if correlation > 0.5:
    print("C'è una correlazione positiva significativa tra i prezzi di iPhone 16 e iPhone 15.")    
else:
    print("Non è presente una correlazione significativa tra i prezzi di iPhone 16 e iPhone 15.") 

        Data  Prezzo_16  Giorni_dal_lancio Trimestre  Prezzo_15
0 2024-09-16      979.0                  3        T1      979.0
1 2024-09-17      979.0                  4        T1      979.0
2 2024-09-18      979.0                  5        T1      979.0
3 2024-09-19      979.0                  6        T1      979.0
4 2024-09-20      979.0                  7        T1      979.0

Correlazione iPhone 16 vs iPhone 15 shiftato: 0.930.
C'è una correlazione positiva significativa tra i prezzi di iPhone 16 e iPhone 15.


#### 2. Analisi

##### 2.1. Preparazione all'analisi
Test di stazionarità, Ricerca dei migliori parametri, Cross-Validation e modello SARIMAX sul test set (hold-out) confrontando MSE e RMSE

In [12]:
#########################
#   TEST STAZIONARITA'
#########################

# Test ADF: X controllare se la serie endog è stazionaria
result = adfuller(merged['Prezzo_16'])
print('ADF Statistic:', round(result[0],4))
print('P-value:', round(result[1],4))

# Ipotesi nulla (H₀): la serie NON è stazionaria → ha trend o stagionalità persistente.
# Ipotesi alternativa (H₁): la serie È stazionaria → media, varianza e autocorrelazione costanti nel tempo.
if result[1] < 0.05:
    print(" →   La serie è stazionaria in quanto il p-value è minore di 0.05")
else:
    print(" →   La serie non è stazionaria in quanto il p-value è maggiore di 0.05")
    result_diff = adfuller(merged['Prezzo_16'].diff().dropna())
    print('\nTest ADF sulla serie differenziata:')
    print('ADF Statistic (prima differenza):', round(result_diff[0],4))
    print('P-value (prima differenza):', round(result_diff[1],4))
    if result_diff[1] < 0.05:
        print(" →   Serie DIFFERENZIATA stazionaria (p-value < 0.05)")
    else:
        print(" →   Serie DIFFERENZIATA NON stazionaria (p-value >= 0.05)")

ADF Statistic: -1.6583
P-value: 0.4527
 →   La serie non è stazionaria in quanto il p-value è maggiore di 0.05

Test ADF sulla serie differenziata:
ADF Statistic (prima differenza): -10.0214
P-value (prima differenza): 0.0
 →   Serie DIFFERENZIATA stazionaria (p-value < 0.05)


In [13]:
#########################
#   TRAIN E TEST SPLIT
#########################

# Dati endogeni e esogeni x il modello
endog = merged['Prezzo_16']
exog = merged['Prezzo_15']

# Split train/test (es. ultimi 90 giorni per test)
train_size = len(endog) - 90
endog_train, endog_test = endog[:train_size], endog[train_size:]
exog_train, exog_test = exog[:train_size], exog[train_size:]

In [14]:
##################################################
#   AUTO ARIMA: PER TROVARE I PARAMETRI OTTIMALI
##################################################

# Trova i parametri ottimali
model_auto = auto_arima(
    endog_train, exogenous=exog_train,
    seasonal=False, #Non stagionale, se c'è staagionalità impostare su True
    stepwise=True,
    suppress_warnings=True,
    trace=True
)

print(model_auto.summary())

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2026.423, Time=0.00 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2019.680, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2018.742, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2025.191, Time=0.00 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2013.669, Time=0.03 sec


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/pytho

 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.09 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2020.186, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2020.759, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2015.956, Time=0.01 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0] intercept
Total fit time: 0.399 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  248
Model:               SARIMAX(1, 1, 1)   Log Likelihood               -1002.835
Date:                Wed, 20 Aug 2025   AIC                           2013.669
Time:                        12:49:14   BIC                           2027.707
Sample:                             0   HQIC                          2019.321
                                - 248                                         
Covariance Type:                  opg             

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [15]:
#   order=(p,d,q):
# •	p=1 → 1 termine autoregressivo (AR)
# •	d=1 → 1 differenza automatica → la serie viene differenziata internamente
# •	q=1 → 1 termine di media mobile (MA)
print(f"Ordine del modello: {model_auto.order}")

#   sesonal_order= (P, D, Q, S):
# Il parametro seasonal_order in SARIMAX serve per modellare componenti stagionali 
# → cioè pattern che si ripetono regolarmente a intervalli costanti (es. settimanali, mensili, annuali).
# • P: Autoregressivo stagionale (SAR)
# • D: Differenza stagionale
# • Q: Media mobile stagionale (SMA)
# • S: Periodo di stagionalità (es. 12 per dati mensili con pattern annuale)

# Se è 0, 0, 0, 0 singifica che non c'è stagionalità
# Se è 1, 1, 1, 12: Il pattern si ripete ogni 12 periodi (es. 12 mesi se i dati sono mensili)
# Se è 1, 1, 1, 30: se i dati sono giornalieri, singifica c'è una stagionalità mensile
print(f"Ordine stagionale del modello: {model_auto.seasonal_order}")

Ordine del modello: (1, 1, 1)
Ordine stagionale del modello: (0, 0, 0, 0)


In [19]:
###############################################
#   MODELLO SARIMAX: HOLD-OUT test finale
###############################################
# x controllare i parametri di auto_arima e la capcità di prevesione del modello

#parametri di auto_arima, oppure si può fare tuning dei parametri (p,d,q)
order = model_auto.order
#modello SARIMAX 
model = SARIMAX(endog_train, exog=exog_train, order=order)
model_fit = model.fit(disp=False)

pred = model_fit.predict(
    start=train_size, 
    end=len(endog)-1, 
    exog=exog_test)

#MSE e RMSE del Testset
mse = mean_squared_error(endog_test, pred)
print(f'MSE Test, su hold-out: {mse:.2f}')
rmse = np.sqrt(mse)
print(f'RMSE Test, su hold-out: {rmse:.2f} euro')

MSE Test, su hold-out: 385.65
RMSE Test, su hold-out: 19.64 euro


In [18]:
###########################
#   CROSS-VALIDATION 
###########################

#Parametri CV
n_splits = 3
step = 90 # previsone di un trimestre, 3 mesi, 90 gionri circa
errors = []

for i in range(n_splits):
    train_end = train_size + i*step
    test_end = train_end + step
    
    if test_end > len(endog): # Se l'indice supera la lunghezza dei dati, esci dal ciclo
        break
    
    endog_train_cv = endog[:train_end]
    exog_train_cv = exog[:train_end]
    endog_test_cv = endog[train_end:test_end]
    exog_test_cv = exog[train_end:test_end]
    
    model_cv = SARIMAX(endog_train_cv, exog=exog_train_cv, order=order)
    fit_cv = model_cv.fit(disp=False)
    
    pred_cv = fit_cv.predict(start=train_end, end=test_end-1, exog=exog_test_cv)
    mse_cv = mean_squared_error(endog_test_cv, pred_cv)
    errors.append(mse_cv)
    print(f'Split {i+1} - MSE: {mse_cv:.2f}')

print(f'- Errore medio MSE su cross-validation: {np.mean(errors):.4f}')
rmse = np.sqrt(np.mean(errors))
print(f'- RMSE medio su cross-validation: {rmse:.2f} euro')

Split 1 - MSE: 385.65
- Errore medio MSE su cross-validation: 385.6497
- RMSE medio su cross-validation: 19.64 euro


In [20]:
fig = go.Figure()
# Prezzo reale
fig.add_trace(go.Scatter(x=endog_test.index, y=endog_test,mode='lines',name='Prezzo reale iPhone 16',line=dict(color='royalblue')))
# Predizione SARIMAX
fig.add_trace(go.Scatter(x=endog_test.index,y=pred,mode='lines',name='Predizione SARIMAX',line=dict(color='firebrick', dash='dash')))
# Layout
fig.update_layout(title='SARIMAX Forecast (Test Set)',xaxis_title='Data',yaxis_title='Prezzo (€)',
    legend=dict(x=0.01, y=0.99),template='plotly_dark',width=900,height=500)
fig.show()

##### 2.2. Previsione

Ci sono due previsioni, un forecast con regressore costante e un forecast con regressore previsto.

Il primo, usa tutti i dati storici di iPhone 16 (endog) e il prezzo di iPhone 15 (exog) traslato.
Per prevedere i prossimi 90 giorni, ipotizzi che il prezzo dell’iPhone 15 rimanga fisso all’ultimo valore conosciuto.
- Pro: utile se si pensa che il prezzo dell’iPhone 15 sia ormai stabile (o non hai un buon modello per prevederlo).
- Contro: se il prezzo di iPhone 15 non fosse realmente costante, l’assunzione può portare a bias.

Il secondo, prima fa un modello SARIMA separato sul prezzo di iPhone 15, per stimare come evolverà nei prossimi mesi (es. con stagionalità). Poi usa questa serie forecastata come regressore esterno nel SARIMAX di iPhone 16:

`exog_forecast = SARIMA(df15['Prezzo']).forecast()`

`SARIMAX(endog, exog=exog).get_forecast(exog=exog_forecast)`

- Pro: più realistico se il prezzo dell’iPhone 15 è ancora volatile (ad es. continua a scendere col tempo); e riene conto di dinamiche stagionali del regressore.
- Contro: doppia fonte di incertezza: se si sbaglia il forecast dell’exog, si propaga l’errore.

Quando usare uno o l’altro
	•	Se il prezzo di iPhone 15 è già assestato (es. fine ciclo di vita), il regressore costante è più realistico.
	•	Se invece il prezzo di iPhone 15 segue ancora trend o stagionalità, il regressore previsto è più accurato (ma più rischioso da stimare).

In [21]:
# Immaginiamo che il prezzo iPhone 15 rimanga costante agli ultimi valori per 90 giorni futuri
last_exog_value = exog.iloc[-1]
exog_future = np.full(90, last_exog_value)

# Fit modello su tutti i dati disponibili
model_final = SARIMAX(endog, exog=exog, order=order)
model_final_fit = model_final.fit(disp=False)

# Previsione 3 mesi avanti
start_pred = len(endog)
end_pred = start_pred + 89

forecast = model_final_fit.predict(start=start_pred, end=end_pred, exog=exog_future)
# Usa get_forecast per ottenere anche l'intervallo di confidenza
forecast_result = model_final_fit.get_forecast(steps=90, exog=exog_future)

# Estrai media e intervallo di confidenza
mean_forecast = forecast_result.predicted_mean
conf_int = forecast_result.conf_int()

print(mean_forecast.head())
print(conf_int.head())


338    776.049851
339    774.209245
340    773.060885
341    772.344421
342    771.897416
Name: predicted_mean, dtype: float64
     lower Prezzo_16  upper Prezzo_16
338       749.615885       802.483817
339       740.169780       808.248710
340       734.397982       811.723788
341       730.350010       814.338831
342       727.243256       816.551577


In [22]:
# 📌 Caso semplice: exog costante
# Immaginiamo che il prezzo iPhone 15 rimanga costante agli ultimi valori per 90 giorni futuri
future_steps = 90
last_exog = exog.iloc[-1] 
exog_future = np.full(future_steps, last_exog)

# Fit modello su tutti i dati disponibili
model_final = SARIMAX(endog, exog=exog, order=order)
fit_final = model_final.fit(disp=False)

forecast_result = fit_final.get_forecast(steps=future_steps, exog=exog_future)
mean_forecast = forecast_result.predicted_mean #Media delle previsioni
conf_int = forecast_result.conf_int() # Intervallo di confidenza

print("Previsioni medie, prox 5g:\n",mean_forecast.head())
print("\nIntervalli di confidenza, prox 5g:\n",conf_int.head())
print(f'\nPrevisione per i prossimi {future_steps} giorni: {mean_forecast.values[-1]:.2f} €\n')

Previsioni medie, prox 5g:
 338    776.049851
339    774.209245
340    773.060885
341    772.344421
342    771.897416
Name: predicted_mean, dtype: float64

Intervalli di confidenza, prox 5g:
      lower Prezzo_16  upper Prezzo_16
338       749.615885       802.483817
339       740.169780       808.248710
340       734.397982       811.723788
341       730.350010       814.338831
342       727.243256       816.551577

Previsione per i prossimi 90 giorni: 771.16 €



In [23]:
forecast_index = np.arange(len(endog), len(endog) + future_steps)
days_iphone15 = df15['Giorni_dal_lancio'][:len(endog)+future_steps]
fig = go.Figure()

fig.add_trace(go.Scatter(x=endog.index, y=endog, mode='lines', 
                         name='Prezzo storico iPhone 16', line=dict(color='cyan')))

fig.add_trace(go.Scatter(x=days_iphone15, y=df15['Prezzo'][:len(endog)+future_steps], mode='lines', 
                         name='Prezzo storico iPhone 15', line=dict(color='royalblue', dash='dot')))

fig.add_trace(go.Scatter(x=forecast_index, y=mean_forecast, mode='lines', 
                         name='Forecast 90 giorni', line=dict(color='magenta', dash='dash')))

fig.add_trace(go.Scatter(x=np.concatenate([forecast_index, forecast_index[::-1]]), y=np.concatenate([conf_int.iloc[:, 0], conf_int.iloc[:, 1][::-1]]), 
                         fill='toself', fillcolor='rgba(255, 182, 193, 0.3)', line=dict(color='rgba(255,255,255,0)'), 
                         hoverinfo="skip", showlegend=True, name='95% Intervallo Confidenza'))

fig.update_layout(title='Previsione iPhone 16 con banda di confidenza', xaxis_title='Giorni dal lancio', yaxis_title='Prezzo (€)', 
                  template='plotly_dark', width=1000, height=500)

fig.show()

In [25]:
# Modello SARIMA sul regressore esterno (iPhone 15) ---
# Esempio: stagionalità annuale di 365 giorni o mensile di 30, dipende dai dati!
# Se hai giornaliero, una stagionalità di 30 può avere senso per pattern mensili.
future_steps = 180
order_exog = (1, 1, 1)
seasonal_order = (1, 1, 1, 30)  # Esempio stagionalità mensile

model_exog = SARIMAX(df15['Prezzo'], order=order_exog, seasonal_order=seasonal_order)
fit_exog = model_exog.fit(disp=False)
forecast_exog = fit_exog.forecast(steps=future_steps) #forecast del regressore esterno: iPhone 15

# combina exog previsto nel SARIMAX del 16
sarimax_final = SARIMAX(endog, exog=exog, order=order, seasonal_order=seasonal_order)
#exog=df15['Prezzo'][:len(endog)],  # solo parte storica per il fit
fit_final = sarimax_final.fit(disp=False)

sarimax_forecast = fit_final.get_forecast(steps=future_steps, exog=forecast_exog)
mean_forecast = sarimax_forecast.predicted_mean
conf_int = sarimax_forecast.conf_int()

# calcolo offset tra forecast e ultimo valore storico
primo_forecast = forecast_exog.iloc[1]
print(f"Il primo valore predetto di forecast_exog è: {primo_forecast:.4f}")

valore_280 = df15['Prezzo'].iloc[279]
if primo_forecast != valore_280:
    print("\nIl primo valore forecast_exog non è uguale al 280º valore di Prezzo iPhone 15:")
    print(f" - 280° valore di Prezzo iPhone 15: {valore_280:.4f}")
    offset = valore_280 - primo_forecast
    print(" - Con una differenza di:", offset)
    forecast_exog_corrected = forecast_exog + offset  # Correzzione del forecast aggiungendo l'offset
    print(" \n - Forecast exog corretto:")
    print("Giorno  Prezzo(€)")
    print(forecast_exog_corrected.head(3))
else:
    print("Il primo valore forecast_exog coincide con il valore storico, nessuna correzione necessaria.")


Il primo valore predetto di forecast_exog è: 647.6657

Il primo valore forecast_exog non è uguale al 280º valore di Prezzo iPhone 15:
 - 280° valore di Prezzo iPhone 15: 779.0000
 - Con una differenza di: 131.33427990806422
 
 - Forecast exog corretto:
Giorno  Prezzo(€)
703    779.658309
704    779.000000
705    781.939399
Name: predicted_mean, dtype: float64


In [26]:
forecast_index = np.arange(len(endog), len(endog) + future_steps)
days_plot2 = df15['Giorni_dal_lancio'][:len(endog) + future_steps]

fig = go.Figure()

fig.add_trace(go.Scatter(x=endog.index, y=endog, mode='lines', name='Prezzo storico iPhone 16', line=dict(color='cyan', width=2)))
fig.add_trace(go.Scatter(x=days_plot2, y=df15['Prezzo'][:len(endog) + future_steps], mode='lines', name='Prezzo storico iPhone 15', line=dict(color='royalblue', dash='dot', width=2)))
fig.add_trace(go.Scatter(x=forecast_index, y=mean_forecast, mode='lines', name='Forecast iPhone 16', line=dict(color='magenta', width=2)))
fig.add_trace(go.Scatter(x=forecast_index, y=forecast_exog_corrected, mode='lines', name='Forecast iPhone 15 (exog)', line=dict(color='orange', dash='dash', width=2)))
fig.add_trace(go.Scatter(x=np.concatenate([forecast_index, forecast_index[::-1]]),y=np.concatenate([conf_int.iloc[:, 0], conf_int.iloc[:, 1][::-1]]),
                         fill='toself',fillcolor='rgba(173, 216, 230, 0.3)',  # lightblue trasparente
                         line=dict(color='rgba(255,255,255,0)'),hoverinfo='skip',showlegend=True,name='95% Intervallo Confidenza'))

fig.update_layout(title='SARIMAX iPhone 16 con regressore previsto',xaxis_title='Giorni dal lancio',yaxis_title='Prezzo (€)',
                  template='plotly_dark',width=1100,height=500)
fig.show()